# Introducing Drift
Now that we have set up TrustyAI to detect drifts, let's introduce some and see what happens.  
To do this, we will simply send a bunch of requests based on our testing dataset.  
Because we only sent a small size of our training dataset to TrustyAI, there is a good chance that the testing dataset will look significantly different.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import numpy as np
import requests
from fetch_artifacts_from_registry import fetch_artifacts_from_registry

In [ ]:
# Disable some warning messages
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

Make sure to add your test models endpoint as `infer_endpoint` here.

In [ ]:
infer_endpoint = "ENTER-YOUR-INFERENCE-ENDPOINT"

deployed_model_name = "jukebox"

In [ ]:
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

In [ ]:
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "input",
                "shape": [1, 13],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=True)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']


In [ ]:
# Load the scaler and test data
with open("scaler.pkl", 'rb') as handle:
    scaler = pickle.load(handle)

X_test = pd.read_pickle("test_data.pkl")[0][:500]

In [ ]:
# Transform our dataset
X_test_scaled = scaler.transform(X_test)

We are sending 500 samples from our test dataset, so this may take a few minutes.

In [ ]:
# Sending predictions
for i, sample in enumerate(X_test_scaled):
    if i%100==0:
        print(f"{i} samples sent")
    prediction = rest_request(X_test_scaled[i].tolist())

And now it's time to go back to the instruction 🏃💨